# 🤖 Análise de Consenso entre LLMs - Anotação sem LLM Hacking

## 1) Setup e Configuração

In [1]:
import sys
from loguru import logger
import pandas as pd
import asyncio

logger.remove()
logger.add(
    sys.stdout,
    format="<green>{time:HH:mm:ss}</green> | <level>{level: <8}</level> | <level>{message}</level>",
    level="INFO"
)

logger.success("✓ Setup completo")

15:59:19 | SUCCESS  | ✓ Setup completo


## 2) Carregar Dataset

In [2]:
from src.utils.data_loader import load_hf_dataset, load_hf_dataset_as_dataframe, list_available_datasets, get_dataset_info


logger.info("Datasets disponíveis:")
for dataset in list_available_datasets():
    logger.info(f"  - {dataset}")

15:59:19 | Datasets disponíveis:
15:59:19 |   - agnews
15:59:19 |   - mpqa
15:59:19 |   - webkb
15:59:19 |   - ohsumed
15:59:19 |   - acm
15:59:19 |   - yelp_2013
15:59:19 |   - dblp
15:59:19 |   - books
15:59:19 |   - reut90
15:59:19 |   - wos11967
15:59:19 |   - twitter
15:59:19 |   - trec
15:59:19 |   - wos5736
15:59:19 |   - sst1
15:59:19 |   - pang_movie
15:59:19 |   - movie_review
15:59:19 |   - vader_movie
15:59:19 |   - subj
15:59:19 |   - sst2
15:59:19 |   - yelp_reviews
15:59:19 |   - 20ng
15:59:19 |   - medline


In [3]:
dataset_name = "sst1"  # Ajuste conforme necessário

texts, categories, ground_truth = load_hf_dataset(dataset_name)

logger.info(f"Textos: {len(texts)}")
logger.info(f"Categorias: {categories}")
logger.info(f"Ground truth: {'Sim' if ground_truth else 'Não'}")

15:59:19 | Carregando dataset: sst1
15:59:19 | Combinando splits: ['train', 'test']
15:59:27 |   ✓ train: 106695 exemplos
15:59:34 |   ✓ test: 11855 exemplos
15:59:34 | Total combinado: 118550 exemplos
15:59:34 | Categorias extraídas automaticamente: [0, 1, 2, 3, 4]
15:59:34 | Amostra reduzida para 10 exemplos (seed=42)
15:59:34 | Coluna de texto: text
15:59:34 | Ground truth carregado da coluna 'label'
15:59:34 | Textos: 10
15:59:34 | Categorias: [0, 1, 2, 3, 4]
15:59:34 | Ground truth: Sim


In [4]:
# Visualizar amostra
logger.info("Amostra dos textos:")
for i, text in enumerate(texts[:3]):
    logger.info(f"{i+1}. {text[:100]}...")
    if ground_truth:
        logger.info(f"   Label: {ground_truth[i]}")

15:59:34 | Amostra dos textos:
15:59:34 | 1. impostor has a handful of thrilling moments and a couple of good performances , but the movie does n...
15:59:34 |    Label: 1
15:59:34 | 2. the acting in pauline and paulette is good all round , but what really sets the film apart is debrau...
15:59:34 |    Label: 4
15:59:34 | 3. mr. polanski is in his element here : alone , abandoned , but still consoled by his art , which is m...
15:59:34 |    Label: 3


In [5]:
df, categories = load_hf_dataset_as_dataframe(dataset_name)

15:59:34 | Carregando dataset: sst1
15:59:34 | Combinando splits: ['train', 'test']
15:59:40 |   ✓ train: 106695 exemplos
15:59:57 |   ✓ test: 11855 exemplos
15:59:57 | Total combinado: 118550 exemplos
15:59:57 | Categorias extraídas automaticamente: [0, 1, 2, 3, 4]
15:59:57 | Amostra reduzida para 10 exemplos (seed=42)
15:59:57 | Coluna de texto: text
15:59:57 | Ground truth carregado da coluna 'label'
15:59:57 | DataFrame criado com 10 linhas


## 3) Configurar Modelos LLM

### Opção A: Usar apenas parâmetros padrão (temp=0)

In [6]:
from src.llm_annotation_system.annotation.llm_annotator import LLMAnnotator
from src.experiments.base_experiment import EXPERIMENT_CONFIG

DEFAULT_MODELS = EXPERIMENT_CONFIG["default_models"]
PROMPT_TEMPLATE = EXPERIMENT_CONFIG["prompt_template"]

annotator = LLMAnnotator(
    dataset_name=dataset_name,
    categories=categories,
    models=DEFAULT_MODELS,
    prompt_template=PROMPT_TEMPLATE,
    use_langchain_cache=False,
    use_alternative_params=False
)

logger.success(f"✓ Annotator inicializado com {len(annotator.models)} modelos")

15:59:58 | Cache carregado: 17165 entradas
15:59:58 | Cache LangChain DESATIVADO explicitamente
15:59:58 | Template do prompt preparado
15:59:58 | LLMAnnotator inicializado
15:59:58 | Modelos: 2 | Categorias: 5
15:59:58 | ✓ Annotator inicializado com 2 modelos


## 4) Executar Anotação

In [7]:
df

,text,label,label_description
0,impostor has a handful of thrilling moments an...,1,negative sentiment
1,the acting in pauline and paulette is good all...,4,very positive sentiment
2,"mr. polanski is in his element here : alone , ...",3,positive sentiment
3,a cop story that understands the medium amazin...,3,positive sentiment
4,the most horrific movie experience i 've had s...,0,very negative sentiment
5,"massoud 's story is an epic , but also a trage...",2,neutral sentiment
6,the actors do n't inhabit their roles -- they ...,0,very negative sentiment
7,more of a career curio than a major work .\n,2,neutral sentiment
8,"there 's a lot of good material here , but the...",2,neutral sentiment
9,if you saw benigni 's pinocchio at a public pa...,1,negative sentiment


In [8]:
annotator.annotation_engine.template

'You are an expert data annotator with extensive experience in text classification tasks.\n\nYour task is to classify the following text into one of the predefined categories with high precision.\n\n**Instructions:**\n1. Read the text carefully and understand its context\n2. Consider the nuances and implicit meanings\n3. Select the most appropriate category based on the content\n4. Be consistent with your classification criteria\n5. If the text is ambiguous, choose the most likely category based on dominant features\n\n**Available Categories:**\n- 0: very negative sentiment\n- 1: negative sentiment\n- 2: neutral sentiment\n- 3: positive sentiment\n- 4: very positive sentiment\n\n**Text to classify (Opinion / Sentiment):**\n{text}\n\n**Important Guidelines:**\n- Provide ONLY the category number as your response\n- Do not include explanations\n- Be objective and avoid bias\n- Consider edge cases carefully\n- Maintain consistency across similar texts\n\n**Your classification to the opinio

### - Testar anotação única

In [9]:
# Parâmetros de anotação
num_repetitions = 3
text = texts[0]
model = "gemma3-4b"

model_strategy = EXPERIMENT_CONFIG["model_strategy"]
rep_strategy = EXPERIMENT_CONFIG["rep_strategy"]

total_annotation = len(text) * num_repetitions
logger.warning(f"  Modelo: {model}")
logger.warning(f"  Texto: {len(text)}")
logger.warning(f"  Repetições: {num_repetitions}")

annotations = await annotator.annotate_single(
        text=text,
        model=model,
        num_repetitions=num_repetitions,
        use_cache=False,
        rep_strategy=rep_strategy
    )

logger.success("✓ Anotação completa")

annotations

15:59:58 |   Modelo: gemma3-4b
15:59:58 |   Texto: 115
15:59:58 |   Repetições: 3
15:59:58 | ✓ Anotação completa


[1, 1, 1]

### - Anotando dataset completo

In [10]:
num_repetitions = EXPERIMENT_CONFIG["num_repetitions_per_llm"]

df_annotations = await annotator.annotate_dataset(
    texts=texts,
    num_repetitions=num_repetitions,
    use_cache=False,
    model_strategy=model_strategy,
    rep_strategy=rep_strategy
)

logger.success("✓ Anotações completas")
df_annotations

15:59:58 | Iniciando anotação
15:59:58 | Textos: 10 | Modelos: 2 | Repetições: 1
15:59:58 | Total de anotações: 20
Anotando:   0%|          | 0/10 [00:00<?, ?it/s]15:59:58 | [START] gemma3-4b @ 95632.281
15:59:58 | [START] mistral-7b @ 95632.282
15:59:59 | [END]   mistral-7b @ 95632.663 | Δ=0.38s
15:59:59 | [END]   gemma3-4b @ 95632.762 | Δ=0.48s
Anotando:  10%|█         | 1/10 [00:00<00:04,  2.08it/s]15:59:59 | [START] gemma3-4b @ 95632.763
15:59:59 | [START] mistral-7b @ 95632.763
15:59:59 | [END]   mistral-7b @ 95633.144 | Δ=0.38s
15:59:59 | [END]   gemma3-4b @ 95633.262 | Δ=0.50s
Anotando:  20%|██        | 2/10 [00:00<00:03,  2.03it/s]15:59:59 | [START] gemma3-4b @ 95633.263
15:59:59 | [START] mistral-7b @ 95633.264
16:00:00 | [END]   mistral-7b @ 95633.651 | Δ=0.39s
16:00:00 | [END]   gemma3-4b @ 95633.775 | Δ=0.51s
Anotando:  30%|███       | 3/10 [00:01<00:03,  1.99it/s]16:00:00 | [START] gemma3-4b @ 95633.776
16:00:00 | [START] mistral-7b @ 95633.777
16:00:00 | [END]   mistral-7

,text_id,text,gemma3-4b_rep1,gemma3-4b_consensus,gemma3-4b_consensus_score,mistral-7b_rep1,mistral-7b_consensus,mistral-7b_consensus_score
0,0,impostor has a handful of thrilling moments an...,1,1,1.0,1,1,1.0
1,1,the acting in pauline and paulette is good all...,3,3,1.0,3,3,1.0
2,2,"mr. polanski is in his element here : alone , ...",3,3,1.0,3,3,1.0
3,3,a cop story that understands the medium amazin...,2,2,1.0,3,3,1.0
4,4,the most horrific movie experience i 've had s...,1,1,1.0,1,1,1.0
5,5,"massoud 's story is an epic , but also a trage...",1,1,1.0,3,3,1.0
6,6,the actors do n't inhabit their roles -- they ...,1,1,1.0,1,1,1.0
7,7,more of a career curio than a major work .\n,1,1,1.0,1,1,1.0
8,8,"there 's a lot of good material here , but the...",1,1,1.0,1,1,1.0
9,9,if you saw benigni 's pinocchio at a public pa...,1,1,1.0,0,0,1.0


### - Metricas por modelo

In [12]:
df_annotations["ground_truth"] = ground_truth if ground_truth else None
df_metrics = annotator.evaluate_model_metrics(
    df_annotations, 
    ground_truth_col="ground_truth", 
    output_csv=True
)

15:49:35 | Calculando métricas por modelo...
15:49:35 | Total de linhas avaliadas: 10
15:49:35 | Métricas para gemma3-4b: Acc=0.3000, F1=0.1889, Prec=0.1571, Rec=0.3000, Cov=1.0000
15:49:35 | Métricas para mistral-7b: Acc=0.3000, F1=0.1905, Prec=0.1400, Rec=0.3000, Cov=1.0000
15:49:35 | Métricas por modelo salvas em: ..\..\results\sst1\model_metrics.csv
15:49:35 | ✓ Métricas calculadas com sucesso


## 5) Calcular Consenso

In [13]:
from src.llm_annotation_system.consensus.consensus_calculator import ConsensusCalculator
from src.llm_annotation_system.consensus.consensus_evaluator import ConsensusEvaluator
from src.experiments.base_experiment import EXPERIMENT_CONFIG


# Inicializar calculador
calculator = ConsensusCalculator(
    consensus_threshold=EXPERIMENT_CONFIG['consensus'].get('threshold', 0.8),
    default_strategy=EXPERIMENT_CONFIG['consensus'].get('strategy', "majority_vote")
)

analyzer = ConsensusEvaluator(categories=categories, calculator=calculator, output_dir=annotator.results_dir)

df_with_consensus = analyzer.compute_consensus(df_annotations)

15:49:35 | Executando cálculo de consenso interno...
15:49:35 | Calculando consenso...
15:49:35 | Consenso calculado:
15:49:35 |   Alto (≥80%): 7 (70.0%)
15:49:35 |   Médio (60-80%): 0 (0.0%)
15:49:35 |   Baixo (<60%): 3 (30.0%)
15:49:35 |   Problemáticos: 3 (30.0%)
15:49:35 |   Itens que precisam de revisão: 0 (0.0%)
15:49:35 | Cálculo de consenso finalizado.


### - Estatisticas

In [14]:
logger.info("\n📊 Estatísticas de Consenso:")
logger.info(f"  Média: {df_with_consensus['consensus_score'].mean():.2%}")
logger.info(f"  Mediana: {df_with_consensus['consensus_score'].median():.2%}")
logger.info(f"  Desvio padrão: {df_with_consensus['consensus_score'].std():.2%}")

15:49:35 | 
📊 Estatísticas de Consenso:
15:49:35 |   Média: 85.00%
15:49:35 |   Mediana: 100.00%
15:49:35 |   Desvio padrão: 24.15%


### - Distribuição por nível

In [15]:
# Distribuição por nível
levels = df_with_consensus['consensus_level'].value_counts()
logger.info("Distribuição por nível:")
for level, count in levels.items():
    logger.info(f"  {level}: {count} ({count/len(df_with_consensus):.1%})")

15:49:35 | Distribuição por nível:
15:49:35 |   high: 7 (70.0%)
15:49:35 |   low: 3 (30.0%)


## - Visualizar consenso

In [16]:
from src.llm_annotation_system.consensus.consensus_visualizer import ConsensusVisualizer

visualizer = ConsensusVisualizer(output_dir=annotator.results_dir)

In [17]:
visualizer.plot_score_and_levels(
    df_with_consensus=df_with_consensus,
    levels=levels
)

✓ Gráfico salvo: score_and_levels.html


## 7️⃣ Análise Detalhada de Consenso

### - Gerando Report

In [18]:
# Gerar relatório
report = analyzer.generate_consensus_report(
    df=df_with_consensus
)

logger.success("✓ Relatório gerado")

15:49:36 | Gerando relatório completo de consenso...
15:49:36 | Fleiss' Kappa: 0.450 (Moderado)
15:49:36 | Casos problemáticos: 3
15:49:36 | Relatório de consenso gerado com sucesso.
15:49:36 | ✓ Relatório gerado


### - Pairwise_agreement

In [19]:
logger.info("\n📊 Gerando heatmap de concordância...")
visualizer.plot_agreement_heatmap(
    agreement_df=report['pairwise_agreement'],
    title='Matriz de Concordância entre Modelos',
)

15:49:36 | 
📊 Gerando heatmap de concordância...


✓ Heatmap salvo: agreement_heatmap.html


### - Cohens Kappa

In [20]:
logger.info("\n📊 Gerando heatmap de Cohens_Kappa...")
visualizer.plot_kappa_heatmap(
    kappa_df=report['cohens_kappa']
)

15:49:36 | 
📊 Gerando heatmap de Cohens_Kappa...


✓ Heatmap salvo: kappa_heatmap.html


### - Casos problemáticos

In [21]:
# Casos problemáticos
problematic = report.get('problematic_cases')
if problematic is not None and len(problematic) > 0:
    logger.warning(f"\n⚠️  {len(problematic)} casos problemáticos identificados")
    display(problematic)
else:
    logger.success("\n✓ Nenhum caso problemático identificado")

15:49:36 | 
⚠️  3 casos problemáticos identificados


,text_id,text,consensus_score,annotations,entropy
0,3,a cop story that understands the medium amazin...,0.5,"{2: 1, 3: 1}",1.0
1,5,"massoud 's story is an epic , but also a trage...",0.5,"{1: 1, 3: 1}",1.0
2,9,if you saw benigni 's pinocchio at a public pa...,0.5,"{1: 1, 0: 1}",1.0


## 8️⃣ Validação com Ground Truth

In [22]:
if ground_truth:
    accuracy, cls_report, cm = analyzer.evaluate_ground_truth(
        df_with_consensus=df_with_consensus
    )

    visualizer.plot_confusion_matrix(
        cm=cm,
        categories=categories
    )
    
else:
    logger.info("⚠️ Ground truth não disponível – pulando validação")

15:49:36 | 
🎯 Accuracy: 30.00%
15:49:36 | 
Classification Report:


{'0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 2.0}, '1': {'precision': 0.2857142857142857, 'recall': 1.0, 'f1-score': 0.4444444444444444, 'support': 2.0}, '2': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 3.0}, '3': {'precision': 0.5, 'recall': 0.5, 'f1-score': 0.5, 'support': 2.0}, '4': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1.0}, 'accuracy': 0.3, 'macro avg': {'precision': 0.15714285714285714, 'recall': 0.3, 'f1-score': 0.18888888888888888, 'support': 10.0}, 'weighted avg': {'precision': 0.15714285714285714, 'recall': 0.3, 'f1-score': 0.18888888888888888, 'support': 10.0}}


c:\Users\gabri\Documents\GitHub\llm-annotation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

c:\Users\gabri\Documents\GitHub\llm-annotation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

c:\Users\gabri\Documents\GitHub\llm-annotation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



✓ Matriz de confusão salva em: ..\..\results\sst1\graphics\confusion_matrix.html


## 9️⃣ Exportar Resultados

In [23]:
import json
from pathlib import Path

# Criar diretório
results_dir = annotator.results_dir
results_dir = results_dir.joinpath("summary")
results_dir.mkdir(parents=True, exist_ok=True)

# Salvar CSVs
df_with_consensus.to_csv(results_dir / 'dataset_anotado_completo.csv', index=False)
logger.info(f"✓ Salvos: {len(df_with_consensus)} registros")

# Alta confiança
high_conf = df_with_consensus[df_with_consensus['consensus_score'] >= 0.8]
high_conf.to_csv(results_dir / 'alta_confianca.csv', index=False)
logger.info(f"✓ Alta confiança: {len(high_conf)} registros")

# Necessita revisão
low_conf = df_with_consensus[df_with_consensus['consensus_score'] < 0.8]
low_conf.to_csv(results_dir / 'necessita_revisao.csv', index=False)
logger.info(f"✓ Necessita revisão: {len(low_conf)} registros")

# Sumário JSON
summary = {
    'dataset': {
        'name': dataset_name,
        'total_texts': len(texts),
        'categories': categories,
        'has_ground_truth': ground_truth is not None
    },
    'config': {
        'models': DEFAULT_MODELS,
        'total_models': len(annotator.models),
        'use_alternative_params': annotator.use_alternative_params,
        'num_repetitions': num_repetitions,
        'total_annotations': len(texts) * len(annotator.models) * num_repetitions
    },
    'results': {
        'consensus_mean': float(df_with_consensus['consensus_score'].mean()),
        'consensus_median': float(df_with_consensus['consensus_score'].median()),
        'high_consensus': int((df_with_consensus['consensus_level'] == 'high').sum()),
        'medium_consensus': int((df_with_consensus['consensus_level'] == 'medium').sum()),
        'low_consensus': int((df_with_consensus['consensus_level'] == 'low').sum()),
    },
    'metrics': {
        'fleiss_kappa': float(report['fleiss_kappa']),
        'fleiss_interpretation': report['fleiss_interpretation']
    }
}

if ground_truth:
    summary['validation'] = {
        'classification_report': cls_report
    }

with open(results_dir / 'sumario_experimento.json', 'w') as f:
    json.dump(summary, f, indent=2)

logger.success("\n✓ Resultados exportados com sucesso!")

15:49:36 | ✓ Salvos: 10 registros
15:49:36 | ✓ Alta confiança: 7 registros
15:49:36 | ✓ Necessita revisão: 3 registros
15:49:36 | 
✓ Resultados exportados com sucesso!


## 10) Resumo Final

In [24]:
logger.info("\n" + "="*80)
logger.success("RESUMO DO EXPERIMENTO")
logger.info("="*80)

logger.info(f"\n📊 Dataset: {dataset_name}")
logger.info(f"  Textos: {len(texts)}")
logger.info(f"  Categorias: {len(categories)}")

logger.info(f"\n🤖 Configuração:")
logger.info(f"  Modelos base: {len(DEFAULT_MODELS)}")
logger.info(f"  Total modelos: {len(annotator.models)}")
logger.info(f"  Alternative params: {annotator.use_alternative_params}")
logger.info(f"  Repetições: {num_repetitions}")

logger.info(f"\n📈 Consenso:")
logger.info(f"  Média: {df_with_consensus['consensus_score'].mean():.2%}")
logger.info(f"  Fleiss' Kappa: {report['fleiss_kappa']:.3f} ({report['fleiss_interpretation']})")

if ground_truth:
    logger.info(f"\n🎯 Validação:")
    logger.info(f"  Accuracy: {accuracy:.2%}")

logger.info(f"\n📁 Arquivos gerados em: {results_dir}/")

cache_stats = annotator.get_cache_stats()
logger.info(f"\n💾 Cache: {cache_stats['total_entries']} entradas")

logger.success("\n✅ Análise completa!")

15:49:36 | 
15:49:36 | RESUMO DO EXPERIMENTO
15:49:36 | ================================================================================
15:49:36 | 
📊 Dataset: sst1
15:49:36 |   Textos: 10
15:49:36 |   Categorias: 5
15:49:36 | 
🤖 Configuração:
15:49:36 |   Modelos base: 2
15:49:36 |   Total modelos: 2
15:49:36 |   Alternative params: False
15:49:36 |   Repetições: 1
15:49:36 | 
📈 Consenso:
15:49:36 |   Média: 85.00%
15:49:36 |   Fleiss' Kappa: 0.450 (Moderado)
15:49:36 | 
🎯 Validação:
15:49:36 |   Accuracy: 30.00%
15:49:36 | 
📁 Arquivos gerados em: ..\..\results\sst1\summary/
15:49:36 | 
💾 Cache: 17165 entradas
15:49:36 | 
✅ Análise completa!
